### 다음달 예측

In [25]:
import pymysql
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
import json

# MySQL 연결 정보
host = "shcrm-mysql.ct8kg24ymta0.ap-northeast-2.rds.amazonaws.com"
port = 14000
database = "shcrm"
user = "root"
password = "5gXofL1FXPEy0u7"

# SQLAlchemy 연결 URL
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

# SQL 쿼리로 사용금액 데이터 가져오기 (전체 데이터를 가져옴)
query = """
    SELECT date, category, amount
    FROM 사용금액
    ORDER BY date
"""
df = pd.read_sql(query, engine)

# date 컬럼의 첫 번째 값이 'date'인 경우 이를 제거
df = df[df['date'] != 'date']

# 날짜 컬럼을 datetime 형식으로 변환
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# 데이터를 피벗 형태로 변환 (날짜별로 각 계정의 금액을 컬럼으로)
df_pivot = df.pivot_table(index='date', columns='category', values='amount', aggfunc='sum').fillna(0)

# 예측 준비 (각 계정과목에 대해 개별적으로 예측)
predictions = {}
account_categories = df_pivot.columns  # 계정과목 목록

for account in account_categories:
    X = np.array(range(len(df_pivot))).reshape(-1, 1)  # 달 수를 Feature로 사용
    y = df_pivot[account].values
    
    # 데이터가 너무 적으면 예측을 건너뛰기
    if len(y) < 2:
        continue  # 데이터가 부족하면 해당 계정을 건너뜁니다.
    
    # 훈련 및 테스트 데이터 분리
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    # 모델 훈련
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # 다음 달 예측 (현재 마지막 날짜에 한 달을 더함)
    next_month = np.array([[len(df_pivot) + 1]])  # 현재 데이터의 마지막 월 이후를 예측
    predictions[account] = model.predict(next_month)[0]

# 예측 결과를 JSON 형식으로 변환
predicted_data = {"predictions": []}
for account, amount in predictions.items():
    predicted_data["predictions"].append({
        "category": account,
        "predicted_amount": round(amount, 2)
    })

# JSON 출력
predicted_json = json.dumps(predicted_data, ensure_ascii=False, indent=4)
print(predicted_json)


{
    "predictions": [
        {
            "category": "관리비",
            "predicted_amount": 560253.78
        },
        {
            "category": "교육훈련비",
            "predicted_amount": 546711.62
        },
        {
            "category": "교통비",
            "predicted_amount": 522083.6
        },
        {
            "category": "복리후생비",
            "predicted_amount": 542088.54
        },
        {
            "category": "소모품비",
            "predicted_amount": 559578.75
        }
    ]
}


In [24]:
import pymysql
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import json

# MySQL 연결 정보
host = "shcrm-mysql.ct8kg24ymta0.ap-northeast-2.rds.amazonaws.com"
port = 14000
database = "shcrm"
user = "root"
password = "5gXofL1FXPEy0u7"

# SQLAlchemy 연결 URL
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

# SQL 쿼리로 사용금액 데이터 가져오기 (전체 데이터를 가져옴)
query = """
    SELECT date, category, amount
    FROM 사용금액
    ORDER BY date
"""
df = pd.read_sql(query, engine)

# date 컬럼의 첫 번째 값이 'date'인 경우 이를 제거
df = df[df['date'] != 'date']

# 날짜 컬럼을 datetime 형식으로 변환
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# 월별로 그룹화하여 카테고리별 금액의 총합을 계산
df['month'] = df['date'].dt.to_period('M')  # 월별로 그룹화
df_monthly = df.groupby(['month', 'category'])['amount'].sum().reset_index()

# 데이터를 피벗 형태로 변환 (월별로 각 계정의 금액을 컬럼으로)
df_pivot = df_monthly.pivot_table(index='month', columns='category', values='amount', aggfunc='sum').fillna(0)

# 월별 총합을 계산 (모든 카테고리 합)
df_pivot['total'] = df_pivot.sum(axis=1)

# 결과를 JSON 형식으로 변환
result_data = {
    "monthly_totals": []
}

# 월별로 데이터 추가
for index, row in df_pivot.iterrows():
    monthly_data = {
        "month": str(index),  # Period 형식의 month를 문자열로 변환
        "categories": row.drop('total').to_dict(),  # 카테고리별 금액
        "total": row['total']  # 월별 총합
    }
    result_data["monthly_totals"].append(monthly_data)

# JSON 출력
result_json = json.dumps(result_data, ensure_ascii=False, indent=4)
print(result_json)

{
    "monthly_totals": [
        {
            "month": "2023-01",
            "categories": {
                "관리비": 15015920.0,
                "교육훈련비": 21814835.0,
                "교통비": 16300061.0,
                "복리후생비": 16804637.0,
                "소모품비": 19339992.0
            },
            "total": 89275445.0
        },
        {
            "month": "2023-02",
            "categories": {
                "관리비": 16539504.0,
                "교육훈련비": 19069995.0,
                "교통비": 16193043.0,
                "복리후생비": 13786393.0,
                "소모품비": 13194541.0
            },
            "total": 78783476.0
        },
        {
            "month": "2023-03",
            "categories": {
                "관리비": 21653724.0,
                "교육훈련비": 18633701.0,
                "교통비": 14294136.0,
                "복리후생비": 13371104.0,
                "소모품비": 21079973.0
            },
            "total": 89032638.0
        },
        {
            "month": "2023-04",
           